# Workshop CentraleSupélec - CeSIA - Partie 4

- Création : 02/2025 par [Nicolas Guillard](mailto:nicolas.guillar@securite-ia.fr) - bénévole au [CeSIA](https://www.securite-ia.fr/).

Créer en s'inspirant particulièrement de [Générer des noms de villes et communes françaises](https://github.com/alxndrTL/villes) par [Alexandre TL](https://www.youtube.com/@alexandretl)


## Présentation du sujet et Plan

## Indications de travail

Les éléments de ce TP :
- le présent carnet,
  
(et ceux qui seront installés grâce au script `Installation de l'environnement de travail`),
- le répertoire `./utils` et les fichiers contenus,
- le répertoire `./weights` contenant les poids des modèles utiles et ceux produits,
- le répertoire `./images` contenant les illustrations des carnets,
- le fichier de données `./villes.txt`.

## Installation de l'environnement de travail

Le script ci-dessous est destiné à installer les éléments nécessaires au fonctionnement de ce carnet.

In [1]:
import sys
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules

repo = "workshop_cs_202503"
branch = "main"
url_repo = f"https://github.com/nicolasguillard/{repo}/archive/refs/heads/{branch}.zip"
target_dir = (
  "/content"
  if IN_COLAB
  else "."
)
resources = ["utils", "weights", "images", "villes.txt"]

if not Path(f"{target_dir}/utils").exists() :
  print("=== Installation des ressources utiles à ce carnet ===")
  !wget -P {target_dir} {url_repo}
  !unzip {target_dir}/{branch}.zip -d {target_dir}
  for resource in resources:
    !mv {target_dir}/{repo}-{branch}/{resource} {target_dir}/{resource}
  !rm -rf {target_dir}/{repo}-{branch}
  !rm -f {target_dir}/{branch}.zip
  print("=== Terminé ===")

  if IN_COLAB:
    print("--- Rafraichissez au besoin la liste des fichiers à gauche si nécessaire ---")
else:
  print("Il semble que des ressources nécessaires pour ce carnet soient déjà installés :")
  for resource in resources:
    print("\t", f"./{resource}", "présent" if Path(f"{target_dir}/{resource}").exists else "absent")
  print("Pour supprimer les ressources automatiquement installées, utilisez la fonction 'remove_resources()' dans un autre bloc de code.")

def remove_resources():
  !rm -rf {target_dir}/{repo}-{branch}
  for resource in resources:
    !rm -rf {target_dir}/{resource}

Il semble que des ressources nécessaires pour ce carnet soient déjà installés :
	 ./utils présent
	 ./weights présent
	 ./images présent
	 ./villes.txt présent
Pour supprimer les ressources automatiquement installées, utilisez la fonction 'remove_resources()' dans un autre bloc de code.


## Les modules et paramétrages globaux

Tous les modules nécessaires sont importés. A moins d'un besoin spécifique, il n'y aura pas besoin de modifier le bloc de code suivant.

In [2]:
# Modules prédéfinis et tiers
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [3]:
#Modules créés pour le projet
from utils import get_datasets, SOS, EOS, PAD, CityNameDataset
from utils import load_transformer_model, TransformerConfig, LanguageModelForSAE, sample, CharTokenizer
from utils import load_sae, AutoEncoder
from utils import clean_memory

### Device

Sélection du GPU selon l'environnement de travail

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


### Paramétrages

In [5]:
# Retirer la limite du nombre maximal de lignes affichées dans un tableau pandas
pd.set_option('display.max_rows', None) 

In [6]:
# Configurer le thème de seaborn
sns.set_theme(style="whitegrid")

In [7]:
# Paramétrer les graines aléatoires
#pth_rnd_gen_device = torch.Generator(device).manual_seed(42)
if device == "cuda":
    pth_rnd_gen_device = torch.cuda.manual_seed(42)
elif device == "mps":
    pth_rnd_gen_device = torch.mps.manual_seed(42)
pth_rnd_gen_cpu = torch.manual_seed(42)
pth_rnd_gen = pth_rnd_gen_cpu if device == "cpu" else pth_rnd_gen_device

## Interprétabilité

##### Les jeux de données

In [8]:
train_dataset, test_dataset, tokenizer, _ = get_datasets("./villes.txt")

creating vocabulary:   0%|          | 0/46 [00:00<?, ?it/s]

creatind dataset:   0%|          | 0/32926 [00:00<?, ?it/s]

creatind dataset:   0%|          | 0/3659 [00:00<?, ?it/s]

##### Chargement du modèle de langue

In [9]:
d_model = 32 # dimension du modèle
n_heads = 4 # nombre de têtes pour l'attention
n_layers = 1 # nombre de couches
dropout = 0.

batch_size = 64

In [10]:
config = TransformerConfig(
    vocab_size=tokenizer.vocabulary_size(),
    d_model=d_model,
    n_heads=n_heads,
    n_layers=n_layers,
    dropout=dropout,
    max_len=max(train_dataset.max_len, test_dataset.max_len) - 1  # Because X and y : sequence[:-1] and sequence[1:] in dataset
)

filename = "./weights/model_32__4_heads__1_layers.pth" # A modifier selon le contexte
#filename = "`./weights/solutions/model_32__4_heads__1_layers.pth`" # A décommenter selon le contexte

In [11]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  target_dir_drive = '/content/drive/MyDrive'
  if Path(f"{target_dir_drive}/{repo}").exists() :
    filename_drive = filename.replace("/weights/", f"/{repo}/")
    !cp {target_dir_drive}/{filename_drive} {target_dir}/{filename}

In [12]:
model = load_transformer_model(filename, class_model=LanguageModelForSAE, config=config, device=device)

LanguageModelForSAE(
  (embedding): Embedding(46, 32, padding_idx=0)
  (core): Transformer(
    (PE): Embedding(46, 32)
    (in_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): DecoderLayer(
        (attention_norm): RMSNorm()
        (sa): SelfAttentionMultiHead(
          (query_proj): Linear(in_features=32, out_features=32, bias=False)
          (key_proj): Linear(in_features=32, out_features=32, bias=False)
          (value_proj): Linear(in_features=32, out_features=32, bias=False)
          (c_proj): Linear(in_features=32, out_features=32, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp_norm): RMSNorm()
        (mlp): MLP(
          (fc_1): Linear(in_features=32, out_features=128, bias=False)
          (fc_2): Linear(in_features=128, out_features=32, bias=False)
          (fc_3): Linear(in_features=32, out_features=128, bias=False)
          (dropout): Dr

##### Chargement du SAE

In [13]:
act_size = config.d_model
num_features = 4 * config.d_model

In [14]:
filename = "./weights/sae_model_32__4_heads__1_layers.pth" # A modifier selon le contexte
#filename = "./weights/solutions/sae_model_32__4_heads__1_layers.pt" # A décommenter selon le contexte

In [15]:
if IN_COLAB:
  if Path(f"{target_dir_drive}/{repo}").exists() :
    filename_drive = filename.replace("/weights/", f"/{repo}/")
    !cp {target_dir_drive}/{filename_drive} {target_dir}/{filename}

In [16]:
sae = load_sae(filename, act_size=act_size, num_features=num_features, device=device)

AutoEncoder()


### Modifier le comportement en le dirigeant

In [17]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Récupération des valeurs maximum des activations de chaque caractéristique dans le SAE par rapport au jeu de données d'entrainement, afin de bénéficier d'un réfénrenciel pour y appliquer un facteur multiplicateur :

In [18]:
model.eval();
sae.eval();

#min_values_sae = torch.full((sae.num_features, 1), +float('inf'))
max_values_sae = torch.full((sae.num_features, ), -float('inf'))

#Pour chaque élément du jeu de données d'entrainement
for X, _ in tqdm(train_dataloader, total=len(train_dataloader)):
    X = X.to(device)
    # récupération des activations cachées du Transformer
    hidden_acts_transfo = model(X, act=True) # (B, S, d_model)
    # récupération des activations de caractéristiques (features) correspondant
    _, _, features, _, _ = sae(hidden_acts_transfo) # (B, S, n_features)
    
    features = features.to("cpu")

    max_features, _ = features.max(dim=0)
    max_features, _ = max_features.max(dim=0)
    max_values_sae = torch.max(max_values_sae, max_features)

clean_memory(device)

  0%|          | 0/515 [00:00<?, ?it/s]

Affichage de ces valeurs (valeur d'activation maximale par indice du neurone caché dans le SAE, sur le jeu de données):

In [19]:
#print(max_values_sae)
for i, v in enumerate(max_values_sae):
    print(f"{i:3d}: {v:.2f}", end="   " if (i+1)%11 else "\n")

  0: 0.56     1: 0.01     2: 2.09     3: 1.15     4: 0.67     5: 1.01     6: 1.01     7: 0.66     8: 0.90     9: 0.33    10: 0.89
 11: 0.88    12: 0.01    13: 0.47    14: 0.53    15: 0.00    16: 0.51    17: 1.04    18: 0.09    19: 0.56    20: 1.41    21: 0.07
 22: 0.76    23: 0.11    24: 0.59    25: 0.11    26: 0.30    27: 0.66    28: 1.41    29: 0.59    30: 0.77    31: 0.49    32: 0.52
 33: 0.86    34: 0.27    35: 0.43    36: 0.96    37: 1.52    38: 0.49    39: 1.47    40: 1.35    41: 0.05    42: 0.12    43: 0.28
 44: 1.58    45: 0.41    46: 0.04    47: 0.96    48: 0.77    49: 0.38    50: 0.32    51: 0.07    52: 0.54    53: 0.94    54: 0.62
 55: 0.02    56: 0.09    57: 0.63    58: 0.33    59: 0.52    60: 0.04    61: 0.86    62: 0.41    63: 0.05    64: 0.29    65: 0.03
 66: 0.15    67: 0.15    68: 0.62    69: 0.43    70: 0.56    71: 0.76    72: 0.81    73: 0.87    74: 0.43    75: 0.66    76: 1.00
 77: 0.75    78: 0.03    79: 2.12    80: 0.94    81: 0.66    82: 0.58    83: 0.68    84: 0

#### EXERCICES : génération contrôlée

Il s'agit d'appliquer la méthode de contrôle consistant à modifier les valeurs d'activations des neurones cachés dans le SAE, en connaissant les concepts qu'ils représentent, selon ce que l'on en a interprété.

![](https://drive.google.com/uc?id=1savRqhjCV4b36dWcQcEPUxouS1m_iftK)

![Contrôle avec SAE](./images/steering.png)

On remarque que l'erreur correspondant à la différence entre l'activation et sa reconstruction est exploitée dans le calcul du tenseur reconstruit en tenant compte des valeurs de contrôle.

Inspirez-vous largement du code de la fonction `sample()` de la partie 3 pour compléter cette fonction qui va générer des noms de commune en tenant compte des modifications qui sont founies via le tenseur `steering_vector`, contenant des lignes [`ìd neurone`, `modification`], selon la méthode illustrée dans le schéma ci-dessus.

Si le `max_values_sae` est fourni en paramètre, la valeur `modification` sera un facteur multiplicatif de la valeur maximale prise par le neurone d'indice `ìd neurone` présente dans `max_values_sae`.

> Conseil(s) : 
> - exploiter les méthodes d'indexation du type `steering_vector[:, 0]`, pour invoquer la première colonne d'un tenseur;
> - ne pas oublié de bien calculer l'erreur;

> NdA : vous constaterez après quelques tentatives d'utilisation de cette fonction que l'art de l'interprétabilité n'est pas une technique facilement maîtrisable.

<details> 
<summary>Eléments d'énoncé</summary>

```python
def steered_sample(
        model: LanguageModelForSAE,
        sae: AutoEncoder,
        tokenizer: CharTokenizer,
        steering_vector: torch.Tensor,
        prompt: str = "",
        max_values_sae: torch.Tensor = None,
        device="cpu",
        g = torch.Generator(),
        ) -> str:
    """
    Args:
        - model (LanguageModelForSAE) :
        - sae (AutoEncoder,) :
        - tokenizer (CharTokenizer) :
        - steering_vector (torch.Tensor) :
        - prompt (str = "") :
        - max_values_sae (torch.Tensor) :
        - device (str) :
        - g (torch.Generator) :
    """
    ### EXERCICE : compléter ce bloc avec les bonnes instructions 
    # DEBUT DE BLOC
    
    ### EXERCICE : à compléter

    return None ### EXERCICE : remplacer None par les bonnes instructions
    # DEBUT DE BLOC
````
</details>

> Solution(s) :

In [20]:
def steered_sample(
        model: LanguageModelForSAE,
        sae: AutoEncoder,
        tokenizer: CharTokenizer,
        steering_vector: torch.Tensor,
        prompt: str = "",
        max_values_sae: torch.Tensor = None,
        device="cpu",
        g = torch.Generator(),
        ) -> str:
    """
    Args:
        - model (LanguageModelForSAE) :
        - sae (AutoEncoder,) :
        - tokenizer (CharTokenizer) :
        - steering_vector (torch.Tensor) :
        - prompt (str = "") :
        - max_values_sae (torch.Tensor) :
        - device (str) :
        - g (torch.Generator) :
    """
    ### EXERCICE : compléter ce bloc avec les bonnes instructions 
    # DEBUT DE BLOC
    
    ### EXERCICE : à compléter

    idx = torch.tensor(
        [tokenizer.char_to_int[SOS]] + tokenizer(prompt),
        dtype=torch.int32,
        device=device
        ).unsqueeze(0)
    next_id = -1

    while next_id != tokenizer.char_to_int[EOS]:
        # activations cachées dans le Transformer
        hidden_act = model(idx, act=True) # (1, l, d_model)

        # encodage et decodage du SAE
        features = sae.encode(hidden_act) # (1, l, num_features)
        act_reconstruct_1 = sae.decode(features) # (1, l, d_model) # reconstruction sans modification

        # decodage du SAE avec l'encodage (les caractéristiques) forcé
        features[:, :, steering_vector[:, 0]] = \
            max_values_sae[steering_vector[:, 0]] * steering_vector[:, 1].float() # forçage des concepts sur chaque lettre
        act_reconstruct_2 = sae.decode(features) # reconstruction avec modification

        # correction de l'erreur de reconstruction
        error = hidden_act - act_reconstruct_1
        final_act = act_reconstruct_2 + error

        # génération des logits
        logits = model.get_logits_(final_act)

        # calcul des probas pour chaque élément du vocabulaire 
        probs = F.softmax(logits[:, -1, :], dim=-1)
        # tirage au sort en prenant en compte ces probas
        next_id = torch.multinomial(probs, num_samples=1, generator=g).item()
        # concaténation
        idx = torch.cat([idx, torch.tensor(next_id, device=device).view(1, 1)], dim=1)

        if idx.shape[1] > model.config.max_len:
            break
        
    return tokenizer.to_string(idx[0].tolist()) ### EXERCICE : remplacer None par les bonnes instructions
    # DEBUT DE BLOC

Réalisons une génération dirigée de noms de commune, en imposant la valeur maximum du neurone ayant l'activation la plus forte :

In [21]:
max_activation_index = max_values_sae.argmax().item()

steering_vector = torch.tensor([[max_activation_index, 1]], dtype=int, device="cpu")

for i in range(15):
    print(steered_sample(
        model,
        sae,
        tokenizer,
        prompt="la",
        steering_vector=steering_vector.to(device),
        max_values_sae=max_values_sae.to(device),
        device=device,
        g=pth_rnd_gen
        )
    )

labyeaaaaiillie
laiiaeé
lai
layeyas
laiaiiau
laiiaaeii
laie
laiazaye
laieiiei
laie
laiaiaaaae
lafoieiae
laiieaaiiiaaoaameiiliaio
laiayaaa
laiaiieaaiiae


A l'opposé, réalisons une génération dirigée de noms de commune, en neutralisant ce neurone :

In [22]:
steering_vector = torch.tensor([[max_activation_index, 0]], dtype=int, device="cpu")

for i in range(15):
    print(steered_sample(
        model,
        sae,
        tokenizer,
        prompt="la",
        steering_vector=steering_vector.to(device),
        max_values_sae=max_values_sae.to(device),
        device=device,
        g=pth_rnd_gen
        )
    )

la stre
la card
la zielle-sorson-ver
la cyande-côff
la cure
la belle-sousse
laboffrinac
lapezoll
larx
lantjoult
la veyès
la geussie-audrain
la crefillec
la quevière-vergolz
la bresque


Comparons avec une génération non dirigée :

In [23]:
for i in range(15):
    print(sample(
        model,
        tokenizer,
        prompt="la",
        device=device,
        g=pth_rnd_gen
        )
    )

la tebray-sur-le-fougneur
la critot
la partiercy
lambirukiry-en-mus
lachen
la bousson
larot
la pigjac
lafougne
la foussin
lairouvesq
lagax
lac-sartières-neneil
laaistey
la hapcap-de-aiterreud-calares
